In [1]:
import pandas as pd
from datasets import load_dataset, Dataset
import os
import re
import torch
from sklearn.model_selection import train_test_split


### PRE-PROCESSING

Per ottenere un dataset che possa essere utilizzato come training del nostro language model partiamo dai file conllu che contengono i testi annotati di wikipedia italiana. Da questi file vogliamo ottenere una struttura dati che per ogni frase riporti id, testo e indice di Gulpease (per ora).

In [2]:
#si ottengono i path di ogni file per il pretraining e si salvano in una lista
ds_directory = "data"
ds_files = []
for file_name in os.listdir(ds_directory):
    file_path = os.path.join(ds_directory, file_name)
    ds_files.append(file_path)
print(ds_files)

['data\\prova.conllu', 'data\\prova2.conllu', 'data\\text_all.txt']


In [3]:
#funzione che calcola gulpease - io in teoria userò read-it
def comp_gulpease(ns, nw, nl):
    g_value = 89 + ((300*ns - 10*nl)/nw) #è corretta questa formula?
    return g_value


In [4]:
#funzione che legge i file conllu riga per riga, estraendo id, testo e indice di complessità di ciascuna frase 
def extract(file_path, n_file):
    id_list = []
    text_list = []
    gulp_list = []
    current_id=""
    current_sent=""
    for line in open(file_path, 'r', encoding='utf-8'):
        if line.startswith("# text"):
            current_sent = line[9:].rstrip('\n')
            words = current_sent.split()
            gulp = comp_gulpease(1, len(words), sum(len(word) for word in words))
            text_list.append(current_sent)
            gulp_list.append(gulp)
        elif line.startswith("# sent_id"):
            current_id = re.sub(r'\D', '', line)
            id_list.append(f'{current_id}_{str(n_file)}') #per avere id univoco ho aggiunto numero del file
    return id_list, text_list, gulp_list
            
            


In [5]:
#si estraggono id e testo tramite le funzioni sopra definite
n_file = 1
id_list = []
text_list = []
gulp_list = []
for item in ds_files:
    item_ids, item_texts, item_gulp = extract(item, n_file)
    id_list = id_list + item_ids
    text_list = text_list + item_texts
    gulp_list = gulp_list + item_gulp
    n_file += 1

In [6]:
#si crea un dataframe con una riga per frase, attributi: id, testo e indice di gulpease
ds_df = pd.DataFrame(columns=["id", "text", "gulp_index"])

ds_df["id"] = id_list
ds_df["text"] = text_list
ds_df["gulp_index"]  = gulp_list

ds_df.head()


,id,text,gulp_index
0,1_1,L'allunaggio è la discesa di un veicolo sulla ...,74.555556
1,2_1,"Si distingue tra allunaggio duro, cioè un impa...",42.703704
2,3_1,"Il programma Luna, partito nel 1959 con la son...",56.142857
3,4_1,"Luna 9, il 3 febbraio 1966, eseguì il primo at...",63.615385
4,5_1,"Il primo allunaggio di un essere umano, il 20 ...",45.000000


In [8]:
ds_df.to_csv("data/data_csv.csv", index=False)